# Training cycle debugging

Basically run code copied from train_seq2seq.py in this notebook to catch bugs

In [1]:
import os
os.environ['ALFRED_ROOT'] = '/home/hoyeung/alfred/'

import sys
sys.path.append(os.path.join(os.environ['ALFRED_ROOT']))
sys.path.append(os.path.join(os.environ['ALFRED_ROOT'], 'models'))

# from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
import torch
import pprint
import json
from data.preprocess import Dataset
from importlib import import_module, reload
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
from models.utils.helper_utils import optimizer_to

In [5]:
# import these if we want to debug a model such as seq2seq_nl_baseline.py

reload(nn)

# from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
import torch
import pprint
import json
from data.preprocess import Dataset
from importlib import import_module, reload
from argparse import ArgumentDefaultsHelpFormatter, ArgumentParser
from models.utils.helper_utils import optimizer_to

<module 'torch.nn' from '/opt/conda/envs/pytorch-py3.6/lib/python3.6/site-packages/torch/nn/__init__.py'>

# Parser

In [6]:
# These are the default flags present in train_seq2seq.py

parser = ArgumentParser(formatter_class=ArgumentDefaultsHelpFormatter)
args = parser.parse_args('')

# settings
args.seed = 123
args.data = 'data/json_feat_2.1.0'
args.splits = 'data/splits/oct21.json'
args.preprocess = False #!
args.pp_folder = 'pp'
args.save_every_epoch = False #!
args.model = 'seq2seq_nl_baseline'
args.gpu = True
args.dout = 'exp/model:seq2seq_nl_baseline'
args.resume = False #!

# hyper parameters
args.batch = 8
args.epoch = 20
args.lr = 1e-4
args.decay_epoch = 10
args.dhid = 512
args.dframe = 2500
args.demb = 100
args.pframe = 300
args.mask_loss_wt = 1.
args.action_loss_wt = 1.
args.subgoal_aux_loss_wt = 0.
args.pm_aux_loss_wt = 0.

# dropouts
args.zero_goal = False #!
args.zero_instr = False #!
args.act_dropout = 0.
args.lang_dropout = 0.
args.input_dropout = 0.
args.vis_dropout = 0.3
args.hstate_dropout = 0.3
args.attn_dropout = 0.
args.actor_dropout = 0.
args.word_dropout = 0.

# other settings
args.dec_teacher_forcing = False #!
args.temp_no_history = False #!

# debugging
args.fast_epoch = False #!
args.dataset_fraction = 0

In [ ]:
# overwrite the default flags

args.model = 'seq2seq_nl_baseline'  # found under models/model/ directory
args.dout = 'exp/model:seq2seq_nl_baseline'

args.dec_teacher_forcing = True
args.gpu = True

# light setup for debugging
args.fast_epoch = True
args.epoch = 5

In [7]:
torch.manual_seed(args.seed)

# Setup and load data splits

In [8]:
# make output dir
if not os.path.isdir(args.dout):
    os.makedirs(args.dout)

print('args.out ', args.dout)

args.out  exp/model:seq2seq_nl_baseline


In [10]:
# load train/valid/tests splits
with open(args.splits) as f:
    splits = json.load(f)
    pprint.pprint({k: len(v) for k, v in splits.items()})

{'tests_seen': 1533,
 'tests_unseen': 1529,
 'train': 21023,
 'valid_seen': 820,
 'valid_unseen': 821}


In [11]:
# preprocess and save -- only need to preprocess once
if args.preprocess:
    print("\nPreprocessing dataset and saving to %s folders ... This will take a while. Do this once as required." % args.pp_folder)
    dataset = Dataset(args, None)
    dataset.preprocess_splits(splits)
    vocab = torch.load(os.path.join(args.dout, "%s.vocab" % args.pp_folder))
else:
    vocab = torch.load(os.path.join(args.data, "%s.vocab" % args.pp_folder))
    
print(vocab)

{'word': Vocab(2360), 'action_low': Vocab(15), 'action_high': Vocab(93)}


# Model and Training

In [12]:
# load the model architecture

M = import_module('model.{}'.format(args.model))
reload(M)
model = M.Module(args, vocab)
optimizer = None

if args.gpu:
    model = model.to(torch.device('cuda'))
    if not optimizer is None:
        optimizer_to(optimizer, torch.device('cuda'))

In [13]:
# examine model layers

model

Module(
  (emb_word): Embedding(2360, 100)
  (emb_action_low): Embedding(15, 100)
  (enc): LSTM(100, 512, batch_first=True, bidirectional=True)
  (enc_att): SelfAttn(
    (scorer): Linear(in_features=1024, out_features=1, bias=True)
  )
  (dec): LanguageDecoder(
    (emb): Embedding(2360, 100)
    (cell): LSTMCell(1124, 1024)
    (attn): DotAttn()
    (input_dropout): Dropout(p=0.0, inplace=False)
    (attn_dropout): Dropout(p=0.0, inplace=False)
    (hstate_dropout): Dropout(p=0.3, inplace=False)
    (word_dropout): Dropout(p=0.0, inplace=False)
    (word): Linear(in_features=2148, out_features=100, bias=True)
    (h_tm1_fc): Linear(in_features=1024, out_features=1024, bias=True)
  )
  (act_dropout): Dropout(p=0.0, inplace=True)
)

In [16]:
# main training loop -- debug here if breakpoints were inserted
model.run_train(splits, optimizer=optimizer)

batch:   0%|          | 0/2 [00:00<?, ?it/s]

Saving to: exp/model:seq2seq_nl_baseline



batch: 100%|██████████| 2/2 [00:00<00:00,  3.35it/s]

batch: 100%|██████████| 2/2 [00:00<00:00,  5.81it/s]

batch: 100%|██████████| 2/2 [00:00<00:00,  5.73it/s]



Found new best valid_seen!! Saving...
> /home/hoyeung/alfred/models/model/seq2seq.py(268)make_debug()
-> return debug
(Pdb) print(debug)
{'trial_T20190909_115736_122556': {'lang_goal': 'Place a cooked potato slice in the sink', 'lang_instr': ['turn to the right twice and to to the end of the counter top and turn to the left and go to the end of the counter top', 'take the knife from the counter top', 'turn to the left and go to the front of the refrigerator and turn to the left and go to the refrigerator', 'open the refrigerator door and slice up the potato on the shelf with the knife and close the refrigerator door', 'turn to the right twice and take a few steps and turn to the left and go to the microwave', 'open the microwave door and put the knife in it and close the microwave door', 'turn to the left twice and take a few step and turn to the right and go to the refrigerator', 'open the refrigerator door and take a potato slice from the shelf and close the refrigerator door', 'tur

(Pdb) for k in debug: print(k)
trial_T20190909_115736_122556
trial_T20190909_091246_807206
trial_T20190918_184236_557252
trial_T20190908_052007_212776
trial_T20190907_232225_725376
trial_T20190908_065238_500229
trial_T20190908_192636_561572
trial_T20190908_165525_911839
trial_T20190911_131350_027076
trial_T20190907_151802_277016
trial_T20190909_010644_297017
trial_T20190909_011522_113515
trial_T20190907_183137_838565
trial_T20190907_033843_707544
trial_T20190910_173916_331859
trial_T20190907_164342_432289
(Pdb) c
Found new best valid_unseen!! Saving...
> /home/hoyeung/alfred/models/model/seq2seq.py(268)make_debug()
-> return debug
(Pdb) q


epoch:   0%|          | 0/2 [01:10<?, ?it/s]


BdbQuit: 

# Look at debugging outputs

In [ ]:
import subprocess

subprocess.call(['ls -l {}'.format(args.dout)])

In [18]:
with open(os.path.join(args.dout, 'valid_seen.debug.preds.json'), 'r') as f:
    debug_tr = json.load(f)

In [19]:
print(type(debug_tr))
print(len(debug_tr))

dict

In [20]:
# all the task numbers
debug_tr.keys()
# let's look at one of them below
task_num = 'trial_T20190909_115736_122556'

dict_keys(['trial_T20190909_115736_122556', 'trial_T20190909_091246_807206', 'trial_T20190918_184236_557252', 'trial_T20190908_052007_212776', 'trial_T20190907_232225_725376', 'trial_T20190908_065238_500229', 'trial_T20190908_192636_561572', 'trial_T20190908_165525_911839', 'trial_T20190911_131350_027076', 'trial_T20190907_151802_277016', 'trial_T20190909_010644_297017', 'trial_T20190909_011522_113515', 'trial_T20190907_183137_838565', 'trial_T20190907_033843_707544', 'trial_T20190910_173916_331859', 'trial_T20190907_164342_432289'])

In [29]:
# We log these from each task
debug_tr[task_num].keys()

dict_keys(['lang_goal', 'lang_instr', 'word_inp_goal', 'word_inp_instr', 'num_inp_goal', 'num_inp_instr', 'action_low', 'action_high', 'p_lang_instr'])

In [50]:
# Gold goal description
debug_tr[task_num]['lang_goal']

'Put two spray bottles on a toilet tank.'

In [51]:
# Gold low-level instruction description
debug_tr[task_num]['lang_instr']

['Turn left, go straight, turn left to face the blue bin next to the toilet, look up.',
 'Take the purple spray bottle from the cabinet above the toilet.',
 'Go backwards while facing towards the blue bin.',
 'Put the spray bottle on the left side of the toilet tank.',
 'Go forward to the blue bin, turn right, look up.',
 'Take the purple spray bottle from the cabinet above that is first from the right.',
 'Turn to face the blue bin. Go backwards while facing towards the blue bin.',
 'Put the spray bottle on the center of the toilet tank.']

In [52]:
# Gold Input low-level action description
debug_tr[task_num]['action_low']

['LookDown_15',
 'RotateLeft_90',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'MoveAhead_25',
 'RotateLeft_90',
 'LookUp_15',
 'LookUp_15',
 'LookUp_15',
 'LookUp_15',
 'LookUp_15',
 'OpenObject',
 'PickupObject',
 'CloseObject',
 'LookDown_15',
 'LookDown_15',
 'LookDown_15',
 'LookDown_15',
 'LookDown_15',
 'RotateLeft_90',
 'RotateLeft_90',
 'MoveAhead_25',
 'MoveAhead_25',
 'RotateRight_90',
 'RotateRight_90',
 'LookUp_15',
 'LookUp_15',
 'PutObject',
 'LookDown_15',
 'LookDown_15',
 'MoveAhead_25',
 'MoveAhead_25',
 'RotateRight_90',
 'LookUp_15',
 'LookUp_15',
 'LookUp_15',
 'LookUp_15',
 'OpenObject',
 'PickupObject',
 'CloseObject',
 'LookDown_15',
 'LookDown_15',
 'LookDown_15',
 'LookDown_15',
 'RotateRight_90',
 'MoveAhead_25',
 'MoveAhead_25',
 'RotateRight_90',
 'RotateRight_90',
 'LookUp_15',
 'LookUp_15',
 'PutObject']

In [53]:
# Gold Input high-level subgoal action description
debug_tr[task_num]['action_high']

['GotoLocation',
 'PickupObject',
 'GotoLocation',
 'PutObject',
 'GotoLocation',
 'PickupObject',
 'GotoLocation',
 'PutObject',
 'NoOp']

In [54]:
# Prediction result
" ".join(debug_tr[task_num]['p_lang_instr'])

['the',
 'left',
 'and',
 'family',
 'to',
 'to',
 'to',
 'left',
 'to',
 'the',
 'the',
 'toilet',
 'to',
 'coach',
 'the',
 'the',
 'toilet',
 '.',
 'of',
 'drop',
 'the',
 'turn',
 'the',
 'toilet',
 'edge',
 'bottle',
 'the',
 'the',
 'toilet',
 '.',
 'the',
 'toilet',
 '.',
 'the',
 'the',
 'the',
 'the',
 'the',
 'the',
 'toilet',
 'to',
 'coach',
 'turn',
 'the',
 'toilet',
 'bottle',
 'the',
 'the',
 'toilet',
 '.',
 'transfer',
 'the',
 'toilet',
 '.',
 'coach',
 'turn',
 'to',
 'the',
 'the',
 'toilet',
 'right',
 'coach',
 'family',
 'left',
 'and',
 'family',
 'to',
 'the',
 'turn',
 'the',
 'toilet',
 'edge',
 'bottle',
 'the',
 'the',
 'toilet',
 '.',
 'the',
 'the',
 'the',
 'the',
 'the',
 'toilet',
 'of',
 'turn',
 'left',
 'the',
 'the',
 'toilet',
 'right',
 'coach',
 'turn',
 'the',
 'to',
 'the',
 'to',
 'the',
 'toilet',
 'to',
 'coach',
 'turn',
 'the',
 'toilet',
 'bottle',
 'the',
 'the',
 'toilet',
 'the',
 'the',
 'toilet',
 '.',
 'handbooks',
 'turn',
 'and'